# General graphing and curve fitting in python.

The purpose of this python notebook is as a quick-start guide in graphing and curving fitting for the student who has not completed the python laboratory classes from PHYS201 before starting the PHYS202 laboratory classes.

To begin with, you need to import the python packages needed for plotting data and curve fitting.

In [ ]:
#run this cell
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit

%matplotlib inline

Let's assume that you have a set of experimental measurements, $y$, that you suspect depend on another set of experimental measurements $x$. Because these are experimental measurements, they each have estimated uncertainties of $\delta y$ and $\delta x$ respectively.

Edit the cell below to enter your own experimental measurements $y$, $x$, $\delta y$ and $\delta x$. Individual entries need to be separated by commas within an array. Be careful that each Numpy array has the same number of entries as the others.

In [ ]:
y=np.array([3,4.4e2,6.5e4,9.7e6,1.44e9])
delta_y=np.array([1,6,6.5e2,9.7e4,1.44e7])
x=np.array([1,2,3,4,5])
delta_x=np.array([0.1,0.1,0.1,0.1,0.1])

You controlled the $x$ values in the experiment, so $x$ is the independent variable and $y$ depends on $x$, so $y$ is the dependent variable. Traditionally this means that $x$ is plotted on the horizontal axis and $y$ is plotted on the vertical axis.

To plot data points on a set of axes, as well as error bars to show the size of their uncertainties, we use the function "errorbar" from matplotlib.

Read the comments in the next cell before running it to understand what is happening at each stage of the plotting.

In [ ]:
#plotting data points as red dots and their error bars as black lines. Don't join the data points with a line.
plt.errorbar(x,y,xerr=delta_x, yerr=delta_y,c="r",marker=".",ecolor="k",ls="none",label="Expt data")
#add x axis label
plt.xlabel("$x$")
#add y axis label
plt.ylabel("$y$",rotation="horizontal")
#add title at the top of the graph
plt.suptitle("$y$ as a function of $x$")
#add a legend formed from the label you gave in "errorbar" command
plt.legend()

Once you have plotted your experimental data, hopefully you'll see some sort of trend between the $y$ coordinates and the $x$ coordinates. Given your general knowledge of the shape of functions like exponentials, power curves, sinusoidal functions (sine, cosine, etc.) and polynomials (straight line, parabola, cubic, etc.), what function do you think could make a good fit?

## A polynomial? 

If you suspect there's a polynomial relationship between the two, then you can use a Numpy function called polyfit to determine the best fitting parameters of a polynomial to your data. You, however, need to specify the degree of the polynomial you think is appropriate; degree 1 is a straight line, degree 2 is a quadratic, degree 3 is a cubic, etc.

Polyfit will determine fitted parameters for your polynomial of choice, and return them in a 1-d array, the parameter associated with the highest degree term first.

When fitting polyfit takes into account the uncertainties in the $y$ data and weights each data point's influence on the fit using the tag w. By setting w=1/delta_y it will place more importance on data points with less uncertainty and less importance on data points with greater uncertainty.

Polyfit can also return a covariance matrix if you set the flag cov="True" which can be used to determine the uncertainty in fitted parameters. However it won't be able to compute a covariance matrix if you don't give it enough data. For a degree $n$ polynomial, you need to give it more than $n+1$ data points, otherwise it's exactly determined with no uncertainty. It may also have problems if your data is a perfect polynomial fit - again there'll be no uncertainty in the fitted parameters. However this is very unlikely to happen when you are fitted real experimental data.

Assuming there is no correlation between the uncertainties in the fitted parameters, you can estimate the uncertainty in each by taking the diagonal elements of the covariance matrix (which correspond to the variance in each), then take the square root of them to get to the standard deviation. You can either express uncertainty in the fitted parameters by their standard deviation, or assuming they follow a Normal distribution, you can say with 95% confidence that the real value of the parameters lies within $\pm 1.96\sigma\approx 2\sigma$ and so express uncertainty as two times the standard deviation.

Edit the commands below to suit your polynomial fitting.

In [ ]:
#edit the degree of the polynomial to a suitable value. Currently it is set to fit a straight line.
param,cov=np.polyfit(x,y,deg=1,cov="True",w=1/delta_y)
#calculating standard deviation of fitted parameters
sd_param=np.sqrt(np.diag(cov))
#print out fitting parameters and their uncertainty in terms of +/- 2 standard deviations.
print("fitted polynomial parameters are:")
print(param)
print("uncertainty in fitted polynomial parameters are:")
print(2*sd_param)

You can add a plot of your fitted polynomial to your graph of experimental results in a few steps. 

First you generate a set of points on the fitted polynomial using Numpy's polyval function, as shown below.

It is helpful to produce a string of its actual equation that can be used as a label in the graph's legend.

Then you plot it with a regular plot function. That's what we've done below. 

In [ ]:
#generating a set of y values on fitted polynomial
fpy=np.polyval(param,x)
#make a string containing the equation of this polynomial - edit if your polynomial is not a straight line!
fplabel="y={:.2E}x+{:.2E}".format(param[0],param[1])
#replot experimental data as before
plt.errorbar(x,y,xerr=delta_x, yerr=delta_y,c="r",marker=".",ecolor="k",ls="none",label="Expt data")
#plot fitted polynomial just as a line and include its label. We've also chosen to make the line green, but that's optional
plt.plot(x,fpy,'g',label=fplabel)
#add x axis label
plt.xlabel("$x$")
#add y axis label
plt.ylabel("$y$",rotation="horizontal")
#add title at the top of the graph
plt.suptitle("$y$ as a function of $x$")
#add a legend formed from the label you gave in "errorbar" command
plt.legend()
#you can save this figure to an external pdf file that you can print out later by the following command.
#just change the filename to something that is meaningful.
plt.savefig("myplot.pdf",dpi=300,orientation="landscape")

## Some other function than a polynomial? 

If you suspect the function that best fits your data is something other than a polynomial; an exponential, a power curve or a sinusoidal function, for example, you can use a curve fitting function from the scipy.optimize package, which we imported in the first cell of this notebook as "curve_fit". 

This "curve_fit" needs you to give it the actual function you want to fit to your data, so you need to start by defining that function, along with its associated parameters.

Below we give examples of three functions - one for a general exponential of the form $y=A\exp{kx}+b$, where $A, k$ and $b$ are parameters,

one for a general power function of the form $y=Cx^n+b$, where $C, n$ and $b$ are parameters,

and one for a sine function $y=A\sin{\omega x+\phi}+b$, where $A, \omega, \phi$ and $b$ are parameters.


In [ ]:
def myExp(x,A,k,b):
    return A*np.exp(k*x)+b

def myPower(x,C,n,b):
    return C*x**n+b

def myWave(x,A,omega,phi,b):
    return A*np.sin(omega*x+phi)+b


Taking a direct quote from the python notebook "CoupledOscil_fit.ipynb" from PHYS201...

"In a nutshell, any fitting algorithm minimizes the difference between the data points and a given function. The function you use must have a set of free parameters that the algorithm changes in order to minimize the differences.
You typically have to provide the algorithm with a 'starting point' for the parameters, so that the algorithm knows where the expected optimum parameters are. Minimizing nonlinear functions in multidimensional spaces is very tricky and you can easily fall in a 'local minimum', that is a set of parameters which are not optimal, but that are better than any parameters in their neighborhood."

So you need to have a good first guess at what your fitted parameters should be to get sensible results from curve_fit!

In [ ]:
#enter some realistic starting parameter values given your experimental data
#You'll need one value for each parameter in your model; we've got 3 in the exponential model, 
#3 in the power model and 4 in the wave model.
startingParams=(1.9,5.3,0.9)

Take a look to the documentation of the function (just google curve_fit python). The basic information you need to feed curve_fit is the function you want it to fit, the x data and the y data, starting values of the parameters and the standard deviation of the y data.

curve_fit should then return an array of the fitted parameters and a covariance matrix from which you can establish the uncertainty in the fitted parameters. (Alot like polyfit did, but curve_fit didn't fit a polynomial!)

You can then substitute your fitted parameters and x values into your defined function to obtain y values on the fitted curve, as shown below.

In [ ]:
#enter your chosen function for fitting - currently the first argument is set to myExp.
#For the standard deviation in the y values - we have assumed that our uncertainty delta_y is 2 times the standard deviation, 
#like it came from a Normal distribution.
popt,pcov=curve_fit(myExp,x,y,startingParams)
#If this doesn't work revise you starting parameters.
#If it still doesn't work, get someone to check you have made a sensible choice of functions to fit.


#calculating uncertainty in fitted parameters
sd_p=np.sqrt(np.diag(pcov))
#print out fitting parameters and their uncertainty in terms of +/- 2 standard deviations.
print("fitted function parameters are:")
print(popt)
print("uncertainty in fitted function parameters are:")
print(2*sd_p)

Finally you can plot you fitted function along with your experimental data to make a nice graph. It's very similar to what was done in the case of the polynomial function fit.

In [ ]:
#generating a set of y values on fitted function - we chose myExp but you should change it to the function you chose.
fittedy=myExp(x,*popt)
#make a string containing the equation of this function - edit it to match your function!
fflabel="y={:.2E}exp({:.2E}x)+{:.2E}".format(popt[0],popt[1],popt[2])
#replot experimental data as before
plt.errorbar(x,y,xerr=delta_x, yerr=delta_y,c="r",marker=".",ecolor="k",ls="none",label="Expt data")
#plot fitted polynomial just as a line and include its label. We've also chosen to make the line green, but that's optional
plt.plot(x,fittedy,'g',label=fflabel)
#add x axis label
plt.xlabel("$x$")
#add y axis label
plt.ylabel("$y$",rotation="horizontal")
#add title at the top of the graph
plt.suptitle("$y$ as a function of $x$")
#add a legend formed from the label you gave in "errorbar" command
plt.legend()

#you can save this figure to an external pdf file that you can print out later by the following command.
#just change the filename to something that is meaningful.
plt.savefig("myplot.pdf",dpi=300,orientation="landscape")
